<a href="https://colab.research.google.com/github/mzohaibnasir/GenAI/blob/main/05_LangchainProject_MCQGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install openai langchain streamlit python-dotenv PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv # imports key value pairs from .env file and can set them as env variables

In [3]:
from langchain.chat_models import ChatOpenAI   # to access openai api
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback

import PyPDF2

In [4]:
from google.colab import userdata

OPENAIAPIKEY = userdata.get("OPENAIAPIKEY")
# os.getenv("OPENAIAPIKEY")

# or

# load_dotenv() will load all env varibales implicity.. mean no declaring


In [5]:

llm = ChatOpenAI(
    api_key=OPENAIAPIKEY,
    model_name="gpt-3.5-turbo",
    temperature=0.5
)
# llm

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# 1. Design input and output prompts


In [6]:
# RESPONSE FORMAT
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [7]:

TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""



In [8]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],  # vars will be input by user
    template=TEMPLATE
)

2. Use LLMChain to connect LLM and prompt

In [9]:
quiz_chain = LLMChain(
    llm=llm,
    prompt = quiz_generation_prompt,
    output_key="quiz",
    verbose=True
)
# quiz_chain

# 3. prompt to evaluate generated quiz

In [10]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [11]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=['subject', 'quiz'],  # vars will be input by user
    template=TEMPLATE2
)

In [12]:
review_chain = LLMChain(
    llm=llm,
    prompt = quiz_evaluation_prompt,
    output_key="review",
    verbose=True
)
# review_chain

# 4. Connect both chains using sequentialChain

In [13]:
generate_evaluate_chain = SequentialChain(

                                          chains=[quiz_chain, review_chain],
                                          input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
                                          output_variables = ['quiz','review'],
                                          verbose=True,
)
# generate_evaluate_chain

# 5. Getting data

to get data :
1. if you have data in pdf, load the pdf, etc on basis of which quiz will be created

In [14]:
file_path = r'./data.txt'

In [17]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [18]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [19]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)


'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

# 6. Token tracking in Langchain
  How to setup token usage traacking in langchain

In [20]:
NUMBER =5
SUBJECT="machine learning"
TONE = 'simple'

In [21]:
# Anything inside the context manager will get tracked.

with get_openai_callback() as cb:
  response = generate_evaluate_chain(
      {
          "text": TEXT,
          "number": NUMBER,
          "subject":SUBJECT,
          "tone":TONE,
          "response_json":json.dumps(RESPONSE_JSON)
      }
  )

  print(cb)


# generate_evaluate_chain:

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [22]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")




Total Tokens:1479
Prompt Tokens:1068
Completion Tokens:411
Total Cost:0.002424


In [23]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [24]:
print(response.get('quiz'))


{
    "1": {
        "mcq": "What is the scientific study of life called?",
        "options": {
            "a": "Chemistry",
            "b": "Biology",
            "c": "Physics",
            "d": "Geology"
        },
        "correct": "b"
    },
    "2": {
        "mcq": "Which theme in biology explains the unity and diversity of life?",
        "options": {
            "a": "Evolution",
            "b": "Photosynthesis",
            "c": "Mitosis",
            "d": "Respiration"
        },
        "correct": "a"
    },
    "3": {
        "mcq": "What allows organisms to move, grow, and reproduce?",
        "options": {
            "a": "Energy processing",
            "b": "Cell division",
            "c": "Mutation",
            "d": "Osmosis"
        },
        "correct": "a"
    },
    "4": {
        "mcq": "What is the primary unit of life that processes hereditary information?",
        "options": {
            "a": "Organelle",
            "b": "Tissue",
            "c": "

In [25]:
quiz = response.get('quiz')
quiz

'\n{\n    "1": {\n        "mcq": "What is the scientific study of life called?",\n        "options": {\n            "a": "Chemistry",\n            "b": "Biology",\n            "c": "Physics",\n            "d": "Geology"\n        },\n        "correct": "b"\n    },\n    "2": {\n        "mcq": "Which theme in biology explains the unity and diversity of life?",\n        "options": {\n            "a": "Evolution",\n            "b": "Photosynthesis",\n            "c": "Mitosis",\n            "d": "Respiration"\n        },\n        "correct": "a"\n    },\n    "3": {\n        "mcq": "What allows organisms to move, grow, and reproduce?",\n        "options": {\n            "a": "Energy processing",\n            "b": "Cell division",\n            "c": "Mutation",\n            "d": "Osmosis"\n        },\n        "correct": "a"\n    },\n    "4": {\n        "mcq": "What is the primary unit of life that processes hereditary information?",\n        "options": {\n            "a": "Organelle",\n        

In [26]:
quiz = json.loads(quiz)

# 6. Creating dataframe

In [27]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})
quiz_table_data

[{'MCQ': 'What is the scientific study of life called?',
  'Choices': 'a: Chemistry | b: Biology | c: Physics | d: Geology',
  'Correct': 'b'},
 {'MCQ': 'Which theme in biology explains the unity and diversity of life?',
  'Choices': 'a: Evolution | b: Photosynthesis | c: Mitosis | d: Respiration',
  'Correct': 'a'},
 {'MCQ': 'What allows organisms to move, grow, and reproduce?',
  'Choices': 'a: Energy processing | b: Cell division | c: Mutation | d: Osmosis',
  'Correct': 'a'},
 {'MCQ': 'What is the primary unit of life that processes hereditary information?',
  'Choices': 'a: Organelle | b: Tissue | c: Cell | d: Organ',
  'Correct': 'c'},
 {'MCQ': 'How long ago did life on Earth emerge?',
  'Choices': 'a: 1 billion years ago | b: 2.5 billion years ago | c: 3.7 billion years ago | d: 5 billion years ago',
  'Correct': 'c'}]

In [28]:
quiz=pd.DataFrame(quiz_table_data)
quiz

,MCQ,Choices,Correct
0,What is the scientific study of life called?,a: Chemistry | b: Biology | c: Physics | d: Ge...,b
1,Which theme in biology explains the unity and ...,a: Evolution | b: Photosynthesis | c: Mitosis ...,a
2,"What allows organisms to move, grow, and repro...",a: Energy processing | b: Cell division | c: M...,a
3,What is the primary unit of life that processe...,a: Organelle | b: Tissue | c: Cell | d: Organ,c
4,How long ago did life on Earth emerge?,a: 1 billion years ago | b: 2.5 billion years ...,c


In [29]:
quiz.to_csv("machinelearning.csv",index=False)

In [30]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'03_26_2024_06_34_21'